In [26]:
import requests
import json
import pandas as pd
import os
import csv
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.translate import AlignedSent, Alignment
import goslate
import time
from translate import Translator
from difflib import SequenceMatcher
from collections import defaultdict

In [2]:
# Read all the MAG csv datasets
DIR_ROOT = "../data/mag/"
MAG_DICT = {}

for r, d, f in os.walk(DIR_ROOT):
    for file in f:
        if file.endswith(".csv"):
            file_path = os.path.join(r, file)
            print("Reading mag file: ",file_path)
            with open(file_path, 'r') as csv_file:
                MAG_DICT[file] = pd.DataFrame(csv.reader(csv_file))         
                #Convert the first line into columns
                MAG_DICT[file].columns = MAG_DICT[file].iloc[0]
                MAG_DICT[file] = MAG_DICT[file].reindex(MAG_DICT[file].index.drop(0)).reset_index(drop=True)
                MAG_DICT[file].columns.name = None
                
                #Normalize the dataframe
                def aff_val_filter(x):
                    return x.strip() != ""
                
                MAG_DICT[file]["AA.DAfN"] = MAG_DICT[file]["AA.DAfN"].apply(lambda x: list(filter(aff_val_filter, x.split(";;")))) 
                

Reading mag file:  ../data/mag/Digital Scholarship in the Humanities (DSH).csv
Reading mag file:  ../data/mag/International Journal of Digital Humanities.csv
Reading mag file:  ../data/mag/Digital Studies - Le champ numérique.csv
Reading mag file:  ../data/mag/International Journal of Humanities and Arts Computing.csv
Reading mag file:  ../data/mag/Digitális Bölcsészet - Digital Humanities.csv
Reading mag file:  ../data/mag/Frontiers in Digital Humanities.csv
Reading mag file:  ../data/mag/AI _ SOCIETY part 2.csv
Reading mag file:  ../data/mag/Journal of Library Metadata.csv
Reading mag file:  ../data/mag/Journal of Quantitative Linguistics.csv
Reading mag file:  ../data/mag/AI _ SOCIETY.csv
Reading mag file:  ../data/mag/Computers and the Humanities part 2.csv
Reading mag file:  ../data/mag/Journal of Cultural Analytics.csv
Reading mag file:  ../data/mag/Computers and the Humanities.csv
Reading mag file:  ../data/mag/International Journal on Digital Libraries.csv
Reading mag file:  ..

In [ ]:
#TEST the results
# COMMENT this PArt
#MAG_DICT["International Journal of Humanities and Arts Computing.csv"]
#MAG_DICT["Literary and Linguistics Computing part 2.csv"]

In [3]:
ACDH_INST_PATH = "../data/acdh/institutions.json"
with open(ACDH_INST_PATH) as json_file:
    acdh_inst = json.load(json_file)

In [23]:
#all the unique institutions we have 
# Some values should ba manually corrected 
# For instance: 'Consiglio Nazionale delle Ricerche'  _and_ 'Consiglio Nazionale delle Ricerche - CNR' are the same institution
# This goes out of this work

inst_df = pd.DataFrame(acdh_inst)
inst_df = inst_df[["name","id"]]
inst_df.columns = ['acdh_name','id']
inst_df['acdh_name'] = inst_df['acdh_name'].apply(lambda x: x.strip().lower()) 
#inst_df['language'] = inst_df['acdh_name'].apply(lambda x: gs.detect(x))
inst_df['acdh_name_eng'] = inst_df['acdh_name'].apply(lambda x: Translator(from_lang = "autodetect", to_lang="en").translate(x).lower())
inst_df['acdh_name_eng'] = inst_df['acdh_name_eng'].apply(lambda x: '' if 'please select' in x else x)
#inst_df['acdh_tokenized'] = inst_df['acdh_name'].apply(lambda x: list(set(word_tokenize(x)) - set(stopwords.words('english'))) )


In [63]:
# Now we check how many elements are similar to what we have in MAG
ROOT_PATH = '../data/mag_acdh_join/'

In [61]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

for k in MAG_DICT:
    #if already processed move on the next one
    if os.path.exists('../data/mag_acdh_join/'+k.replace(".csv","")+'.json'):
        continue
    print("Join "+k+"with acdh institutions ...")
    new_column = []
    for index, row in MAG_DICT[k].iterrows():
        
        l_aff = []
        for aff in row['AA.DAfN']:    
            # Check inside the ACDH affiliations if present
            flag_found = -1
            for index_acdh, row_acdh in inst_df.iterrows():
                if ((similar(row_acdh["acdh_name"],aff) > 0.8) or (similar(row_acdh["acdh_name_eng"],aff) > 0.8)):
                    flag_found = row_acdh["id"]
                    break
            
            l_aff.append(flag_found)
 
        new_column.append(l_aff)

    # add the new column
    MAG_DICT[k]["acdh_aff"] = new_column
    
    MAG_DICT[k][["AA.DAfN","DOI","acdh_aff"]].to_json(r'../data/mag_acdh_join/'+k.replace(".csv","")+'.json',orient='records')

Join International Journal of Humanities and Arts Computing.csvwith acdh institutions ...
Join Digitális Bölcsészet - Digital Humanities.csvwith acdh institutions ...
Join Frontiers in Digital Humanities.csvwith acdh institutions ...
Join AI _ SOCIETY part 2.csvwith acdh institutions ...
Join Journal of Library Metadata.csvwith acdh institutions ...
Join Journal of Quantitative Linguistics.csvwith acdh institutions ...
Join AI _ SOCIETY.csvwith acdh institutions ...
Join Computers and the Humanities part 2.csvwith acdh institutions ...
Join Journal of Cultural Analytics.csvwith acdh institutions ...
Join Computers and the Humanities.csvwith acdh institutions ...
Join International Journal on Digital Libraries.csvwith acdh institutions ...
Join D-Lib Magazine part 2.csvwith acdh institutions ...
Join D-Lib Magazine.csvwith acdh institutions ...
Join Computational Linguistics.csvwith acdh institutions ...
Join Virtual Archaeology Review.csvwith acdh institutions ...
Join Journal of the J

In [158]:
with open("../data/res/affiliation.json") as json_file:
    affiliation_df = pd.DataFrame(json.load(json_file))

ERRORS = [" Pisa"]
affiliation_df = affiliation_df.set_index('DOI').drop(" Pisa")
affiliation_df.to_json("../data/res/affiliation(with_id).json", orient='index')

In [67]:
## DRAFT

#all_data = []
#for r, d, f in os.walk(ROOT_PATH):
#    for file in f:
#        if file.endswith(".json"):
#            file_path = os.path.join(ROOT_PATH, file)
#            with open(file_path) as json_file:
#                all_data = all_data + json.load(json_file)

#with open(ROOT_PATH+'affiliation.json', 'w') as file:
#     file.write(json.dumps(all_data)) # use `json.loads` to do the reverse